In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [7]:
transform = transforms.Compose([transforms.Resize((227, 227)), transforms.ToTensor()])

train_dataset = datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

test_dataset = datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
class AlexNet(nn.Module):
    def __init__(self, num_classes = 1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size = 11, stride = 4, padding = 0),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(96, 256, kernel_size = 5, padding = 2),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(256, 384, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(384, 384, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(384, 256, kernel_size = 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 3, stride = 2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace = True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [21]:
!pip install torchsummary
!pip install tqdm

In [15]:
import torchsummary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print()

model = AlexNet()
model.to(device)

torchsummary.summary(model, input_size = (3, 227, 227), device = 'cuda')

cuda

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Dropout-14             

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

model = AlexNet(num_classes = 10).to(device)

In [19]:
def train_model(model, train_loader, criterion, optimizer, num_epochs = 5):
    model.train()
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss : {loss.item():.4f}')

train_model(model, train_loader, criterion, optimizer, num_epochs = 5)

Epoch [1/5], Loss : 2.3062
Epoch [2/5], Loss : 2.3024
Epoch [3/5], Loss : 2.3020
Epoch [4/5], Loss : 2.3031
Epoch [5/5], Loss : 2.3026


In [25]:
from tqdm import tqdm  # tqdm 임포트

def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        # tqdm을 사용하여 train_loader의 진행 상황을 표시
        epoch_loss = 0  # 에포크 손실 초기화
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # 에포크 손실 누적

        # 에포크 손실 출력
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss : {epoch_loss / len(train_loader):.4f}')

# 모델 훈련 호출
train_model(model, train_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Loss : 2.3026


Epoch [2/5], Loss : 2.3026


Epoch [3/5], Loss : 2.3026


Epoch [4/5], Loss : 2.3026


Epoch [5/5], Loss : 2.3026
